In [101]:
library(data.table)
library(reticulate)
library(SoupX)
library(Matrix)

scipy_sparse = import("scipy.sparse")
np = import("numpy")

pathin="/project/genomics/lianyun/CaIPACL/cp_from_homepath/data_final4"
pathout="/project/genomics/lianyun/CaIPACL/cp_from_homepath/data_final4"
pathout_plot="/home/hpc/lianyun.huang/singlecell/final4/output/output_soupx"

In [102]:
sample="12"
nforce="4500"

In [103]:
# read data_tod
data_tod=scipy_sparse$load_npz(paste0(pathin,"/data_tod",sample,".npz"))

# read soupx_groups
soupx_groups = fread(paste0(pathin,"/soupx_groups",sample,".csv"))
soupx_groups=data.frame(soupx_groups)
soupx_groups=py_run_string("import pandas as pd", convert = TRUE)$pd$Series(soupx_groups$V2)



In [104]:
# read and format data
data=scipy_sparse$load_npz(paste0(pathin,"/data",sample,".npz"))

cells = fread(paste0(pathin,"/cells",sample,".csv"),header=FALSE)
cells=data.frame(cells)
cells=cells[,c(1)]

genes = fread(paste0(pathin,"/genes",sample,".csv"),header=FALSE)
genes=data.frame(genes)
genes=genes[,c(1)]

rownames(data)=genes
colnames(data)=cells

data=as(data,"sparseMatrix")

In [105]:
# Generate SoupChannel Object
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# add extra meta data to the SoupChannel
soupProf = data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)

# Set cluster information in SoupChannel
sc = setClusters(sc, soupx_groups)

pdf(paste0(pathout_plot,"/soupxrho_sample",sample,".pdf"))
# Estimate contamination fraction
sc  = autoEstCont(sc, doPlot=TRUE)
dev.off()

# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)

47 genes passed tf-idf cut-off and 47 soup quantile filter.  Taking the top 47.

Using 1418 independent estimates of rho.

Estimated global rho of 0.03



png 
  2

Warning message in sparseMatrix(i = out@i[w] + 1, j = out@j[w] + 1, x = out@x[w], :
“'giveCsparse' is deprecated; setting repr="T" for you”
Expanding counts from 34 clusters to 2137 cells.



In [106]:
writeMM(out,paste0(pathout,"/out",sample,".mtx"))

NULL